In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
#from google.colab import drive
import pandas as pd
import yfinance

In [2]:
#drive.mount("/content/drive")

In [2]:
file_path_tda = '../feature_extraction/tda_features.csv' #'/content/drive/My Drive/LSTM/tda_features.csv'
file_path_finance = '../sp500_data.csv' #'/content/drive/My Drive/LSTM/sp500_data.csv'
file_path_ticker = '../sp500.csv' #'/content/drive/My Drive/LSTM/sp500.csv'

In [3]:
df_finance = pd.read_csv(file_path_finance, header=[0,1])
df_finance = df_finance.drop(0)

In [4]:
df_finance

Ticker        NWS                                             \
           Price       Open       High        Low      Close    Volume   
1     2018-01-02  15.250455  15.433644  15.204658  15.387846  463900.0   
2     2018-01-03  15.387846  15.479441  15.342050  15.479441  402500.0   
3     2018-01-04  15.479440  15.644310  15.387845  15.506919  877800.0   
4     2018-01-05  15.754226  15.937413  15.479442  15.754226  473100.0   
5     2018-01-08  15.708428  15.800022  15.571036  15.754226  378900.0   
...          ...        ...        ...        ...        ...       ...   
1922  2025-08-25  35.041681  35.131404  34.543222  34.603035  617400.0   
1923  2025-08-26  34.573128  34.573128  34.114546  34.184330  488200.0   
1924  2025-08-27  34.114544  34.383713  33.955038  34.004883  420100.0   
1925  2025-08-28  34.174359  34.363776  33.785562  33.945068  501900.0   
1926  2025-08-29  33.905189  33.984944  33.596147  33.765621  595800.0   

            TMUS                                      ...         DHR  \
            Open        High         Low       Close  ...        Open   
1      62.040010   62.641958   61.690488   62.175934  ...   79.695788   
2      62.030302   62.263314   61.408932   61.758453  ...   78.891805   
3      62.030304   62.137102   61.049704   61.166210  ...   80.499718   
4      61.690492   62.923521   61.282715   62.719635  ...   80.833319   
5      62.583715   64.059476   62.438080   63.205086  ...   80.927401   
...          ...         ...         ...         ...  ...         ...   
1922  252.211090  252.500064  249.002397  250.855865  ...  210.927390   
1923  250.696419  250.945541  248.544002  250.317749  ...  207.413592   
1924  248.902735  251.383996  248.693482  250.477188  ...  205.277360   
1925  249.839434  250.766161  248.504144  249.679993  ...  204.109432   
1926  249.029999  253.149994  248.860001  251.990005  ...  204.977882   

                                                           WMT             \
            High         Low       Close     Volume       Open       High   
1      79.960923   78.840496   79.062874  4087646.0  29.182010  29.326011   
2      80.320140   78.874699   80.114868  2791236.0  29.049770  29.387727   
3      81.175396   80.414195   80.499718  2397226.0  29.252541  29.323069   
4      81.243856   80.687919   81.004379  2121430.0  29.281930  29.499398   
5      81.517550   80.645153   81.466232  2038522.0  29.461192  29.902008   
...          ...         ...         ...        ...        ...        ...   
1922  211.356624  206.924451  207.363678  1985800.0  97.000000  97.580002   
1923  208.990808  205.307312  205.886292  3670600.0  96.279999  96.470001   
1924  207.024273  203.440608  204.638489  2503200.0  95.900002  96.320000   
1925  205.317290  201.663756  204.927979  2519900.0  96.000000  96.570000   
1926  207.333721  204.339011  205.457047  1979800.0  96.099998  97.029999   

                                        
            Low      Close      Volume  
1     28.952788  28.973360  30451500.0  
2     29.026258  29.226097  26710800.0  
3     29.067399  29.252541  20491200.0  
4     29.273115  29.425930  21852000.0  
5     29.449434  29.860863  26531700.0  
...         ...        ...         ...  
1922  96.070000  96.070000  18956100.0  
1923  95.419998  96.050003  22270400.0  
1924  95.599998  96.080002  12925700.0  
1925  95.699997  96.110001  19241600.0  
1926  95.800003  96.980003  15600300.0  

[1926 rows x 2520 columns]

In [ ]:


df_finance.dropna(axis=1, inplace=True)
df_finance

Ticker        NWS                                             \
           Price       Open       High        Low      Close    Volume   
1     2018-01-02  15.250455  15.433644  15.204658  15.387846  463900.0   
2     2018-01-03  15.387846  15.479441  15.342050  15.479441  402500.0   
3     2018-01-04  15.479440  15.644310  15.387845  15.506919  877800.0   
4     2018-01-05  15.754226  15.937413  15.479442  15.754226  473100.0   
5     2018-01-08  15.708428  15.800022  15.571036  15.754226  378900.0   
...          ...        ...        ...        ...        ...       ...   
1922  2025-08-25  35.041681  35.131404  34.543222  34.603035  617400.0   
1923  2025-08-26  34.573128  34.573128  34.114546  34.184330  488200.0   
1924  2025-08-27  34.114544  34.383713  33.955038  34.004883  420100.0   
1925  2025-08-28  34.174359  34.363776  33.785562  33.945068  501900.0   
1926  2025-08-29  33.905189  33.984944  33.596147  33.765621  595800.0   

            TMUS                                      ...         DHR  \
            Open        High         Low       Close  ...        Open   
1      62.040010   62.641958   61.690488   62.175934  ...   79.695788   
2      62.030302   62.263314   61.408932   61.758453  ...   78.891805   
3      62.030304   62.137102   61.049704   61.166210  ...   80.499718   
4      61.690492   62.923521   61.282715   62.719635  ...   80.833319   
5      62.583715   64.059476   62.438080   63.205086  ...   80.927401   
...          ...         ...         ...         ...  ...         ...   
1922  252.211090  252.500064  249.002397  250.855865  ...  210.927390   
1923  250.696419  250.945541  248.544002  250.317749  ...  207.413592   
1924  248.902735  251.383996  248.693482  250.477188  ...  205.277360   
1925  249.839434  250.766161  248.504144  249.679993  ...  204.109432   
1926  249.029999  253.149994  248.860001  251.990005  ...  204.977882   

                                                           WMT             \
            High         Low       Close     Volume       Open       High   
1      79.960923   78.840496   79.062874  4087646.0  29.182010  29.326011   
2      80.320140   78.874699   80.114868  2791236.0  29.049770  29.387727   
3      81.175396   80.414195   80.499718  2397226.0  29.252541  29.323069   
4      81.243856   80.687919   81.004379  2121430.0  29.281930  29.499398   
5      81.517550   80.645153   81.466232  2038522.0  29.461192  29.902008   
...          ...         ...         ...        ...        ...        ...   
1922  211.356624  206.924451  207.363678  1985800.0  97.000000  97.580002   
1923  208.990808  205.307312  205.886292  3670600.0  96.279999  96.470001   
1924  207.024273  203.440608  204.638489  2503200.0  95.900002  96.320000   
1925  205.317290  201.663756  204.927979  2519900.0  96.000000  96.570000   
1926  207.333721  204.339011  205.457047  1979800.0  96.099998  97.029999   

                                        
            Low      Close      Volume  
1     28.952788  28.973360  30451500.0  
2     29.026258  29.226097  26710800.0  
3     29.067399  29.252541  20491200.0  
4     29.273115  29.425930  21852000.0  
5     29.449434  29.860863  26531700.0  
...         ...        ...         ...  
1922  96.070000  96.070000  18956100.0  
1923  95.419998  96.050003  22270400.0  
1924  95.599998  96.080002  12925700.0  
1925  95.699997  96.110001  19241600.0  
1926  95.800003  96.980003  15600300.0  

[1926 rows x 2381 columns]

In [34]:
df_finance = pd.read_csv(file_path_finance, header=[0,1])
df_finance = df_finance.drop(0)
df_finance.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df_finance.columns]
df_finance = df_finance.rename(columns={"Ticker_Price": "Date"})
df_finance = df_finance.set_index("Date")

df_tda = pd.read_csv(file_path_tda)
df_ticker = pd.read_csv(file_path_ticker)

df_finance.head()

,NWS_Open,NWS_High,NWS_Low,NWS_Close,NWS_Volume,TMUS_Open,TMUS_High,TMUS_Low,TMUS_Close,TMUS_Volume,...,DHR_Open,DHR_High,DHR_Low,DHR_Close,DHR_Volume,WMT_Open,WMT_High,WMT_Low,WMT_Close,WMT_Volume
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,15.250455,15.433644,15.204658,15.387846,463900.0,62.040010,62.641958,61.690488,62.175934,2192400.0,...,79.695788,79.960923,78.840496,79.062874,4087646.0,29.182010,29.326011,28.952788,28.973360,30451500.0
2018-01-03,15.387846,15.479441,15.342050,15.479441,402500.0,62.030302,62.263314,61.408932,61.758453,2610800.0,...,78.891805,80.320140,78.874699,80.114868,2791236.0,29.049770,29.387727,29.026258,29.226097,26710800.0
2018-01-04,15.479440,15.644310,15.387845,15.506919,877800.0,62.030304,62.137102,61.049704,61.166210,2384400.0,...,80.499718,81.175396,80.414195,80.499718,2397226.0,29.252541,29.323069,29.067399,29.252541,20491200.0
2018-01-05,15.754226,15.937413,15.479442,15.754226,473100.0,61.690492,62.923521,61.282715,62.719635,3292600.0,...,80.833319,81.243856,80.687919,81.004379,2121430.0,29.281930,29.499398,29.273115,29.425930,21852000.0
2018-01-08,15.708428,15.800022,15.571036,15.754226,378900.0,62.583715,64.059476,62.438080,63.205086,3606000.0,...,80.927401,81.517550,80.645153,81.466232,2038522.0,29.461192,29.902008,29.449434,29.860863,26531700.0


In [5]:
df_finance.columns.size

2519

In [6]:
"""
betas_array = df_tda[["", 'beta0', 'beta1', 'beta2']]
feature_vectors = []

for i in range(len(df_tda)):
    row = df_tda.iloc[i]
    feature_vector = np.array([
        betas_array[i]      
    ])

    feature_vectors.append(feature_vector)

feature_vectors = np.array(feature_vectors)
"""

'\nbetas_array = df_tda[["", \'beta0\', \'beta1\', \'beta2\']]\nfeature_vectors = []\n\nfor i in range(len(df_tda)):\n    row = df_tda.iloc[i]\n    feature_vector = np.array([\n        betas_array[i]      \n    ])\n\n    feature_vectors.append(feature_vector)\n\nfeature_vectors = np.array(feature_vectors)\n'

In [7]:
"""
symbol_to_security = df_ticker.set_index("Symbol")["Security"].to_dict()

sym=df_ticker["Symbol"].tolist()
tickers = np.array(sym)

length = []
for i in range(len(tickers)):
  length.append(len(symbol_to_security[tickers[i]]))
length=np.array(length)
"""

'\nsymbol_to_security = df_ticker.set_index("Symbol")["Security"].to_dict()\n\nsym=df_ticker["Symbol"].tolist()\ntickers = np.array(sym)\n\nlength = []\nfor i in range(len(tickers)):\n  length.append(len(symbol_to_security[tickers[i]]))\nlength=np.array(length)\n'

In [8]:
tickers = set(col.split("_")[0] for col in df_finance.columns)

In [9]:
window_rsi = 14
window_vol = 21

def compute_rsi(series, window=14):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    roll_up = pd.Series(gain, index=series.index).rolling(window=window).mean()
    roll_down = pd.Series(loss, index=series.index).rolling(window=window).mean()
    RS = roll_up / roll_down
    RSI = 100.0 - (100.0 / (1.0 + RS))
    return RSI

for ticker in tickers:
    close_col = f"{ticker}_Close"
    volume_col = f"{ticker}_Volume"

    if close_col not in df_finance.columns or volume_col not in df_finance.columns:
        continue

    df_finance[f"{ticker}_RSI_14"] = compute_rsi(df_finance[close_col], window=window_rsi)

    log_return = np.log(1 + df_finance[close_col].pct_change())
    df_finance[f"{ticker}_Log_Return"] = log_return
    df_finance[f"{ticker}_Volatility_21"] = log_return.rolling(window=window_vol).std()
    
    df_finance[f"{ticker}_Volume_Z"] = (df_finance[volume_col] - df_finance[volume_col].rolling(window_vol).mean()) / df_finance[volume_col].rolling(window_vol).std()


/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_66398/3442652859.py:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_Volume_Z"] = (df_finance[volume_col] - df_finance[volume_col].rolling(window_vol).mean()) / df_finance[volume_col].rolling(window_vol).std()
/var/folders/c5/nwrhnbhs5ks1ssf190f4b49r0000gn/T/ipykernel_66398/3442652859.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_finance[f"{ticker}_RSI_14"] = compute_rsi(df_finance[close_col], window=window_rsi)
/var/folders/c5/nwrhnbhs5k

In [10]:
df_finance.columns.size

4531

In [ ]:
#extra cleaning Nans

biggest_cut_off = max(window_rsi, window_vol)
df_finance = df_finance[biggest_cut_off:]

In [13]:
for i in range(len(df_finance)):
    row = df_finance.iloc[i]
    nan_cols = row.index[row.isnull()]
    if len(nan_cols) > 0:
        print(f"Row {i} has NaN in columns: {list(nan_cols)}")
